<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Homework - Hyperparameter Tuning</h2>
<h4>DAT-5303 | Machine Learning</h4>

<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

Team: TEAM 2

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<br>
In this homework, your team is tasked with tuning the hyperparameters of two of the following model types:

* Regression Tree
* Random Forest
* Gradient Boosted Machine (GBM)

<h3>Step 1: Imports</h3>
Import the following packages and make sure to start replacing my comments with your own. Also, please tell me about which models you are tuning in the second cell.

In [82]:
######################################## IMPORT PACKAGES ###############################################
import pandas            as pd                           # import pandas to manipulate data
import numpy as np                                       # to use arange function from numpy


# importing model types
from sklearn.ensemble import RandomForestRegressor       # importing package random forest 
from sklearn.ensemble import GradientBoostingRegressor   # importing package gradient boosting regressor

# importing ML tools
from sklearn.model_selection import train_test_split     # import test-train data package
from sklearn.model_selection import RandomizedSearchCV   # import random search package



########################################## LOAD DATASET ###############################################
# load dataset called housing from excel using pandas
housing = pd.read_excel('./Scripts - Regression Modeling/__datasets/housing_feature_rich.xlsx')


######################################## SET OUTPUT VIEW ##############################################
# set output height, width, and number of column
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)


######################################### SET VARIABLES ###############################################
# set variables in list to subset in while using train-test-split

# x_variables for chosen variables
x_variables = ['Garage_Cars', 'Overall_Qual', 'Total_Bsmt_SF',
               'NridgHt', 'Kitchen_AbvGr', 'has_Second_Flr',
               'Mas_Vnr_Area', 'has_Garage', 'Porch_Area',
               'NWAmes', 'OldTown', 'Overall_Cond',
               'Edwards', 'Somerst', 'Fireplaces',
               'Second_Flr_SF', 'First_Flr_SF', 'has_Mas_Vnr',
               'CulDSac', 'Total_Bath', 'Crawfor', 'Garage_Area',
               'has_Porch']

# full_x is for every columns in housing dataset except Sale_Price
full_x = ['Overall_Qual', 'Overall_Cond', 'Mas_Vnr_Area', 'Total_Bsmt_SF',
          'First_Flr_SF', 'Second_Flr_SF', 'Gr_Liv_Area', 'Full_Bath',
          'Half_Bath', 'Kitchen_AbvGr', 'TotRms_AbvGr', 'Fireplaces',
          'Garage_Cars', 'Garage_Area', 'Porch_Area', 'log_Lot_Area',
          'has_Second_Flr', 'has_Garage', 'has_Mas_Vnr', 'has_Porch',
          'Total_Bath', 'CulDSac', 'BrkSide', 'CollgCr', 'Crawfor',
          'Edwards', 'Gilbert', 'Mitchel', 'NWAmes', 'NridgHt', 'OldTown',
          'Sawyer', 'SawyerW', 'Somerst', 'Other_NH']


# reduced_x is for a list of columns name with 
# smaller number of column to run regression model
reduced_x = ['Overall_Qual', 'Gr_Liv_Area', 'Full_Bath',
             'Kitchen_AbvGr', 'TotRms_AbvGr', 'Fireplaces',
             'Garage_Cars', 'Garage_Area', 'Porch_Area', 
             'log_Lot_Area', 'has_Second_Flr', 'has_Garage',
             'has_Mas_Vnr', 'has_Porch', 'Total_Bath', 'CulDSac']



### verify imported dataset ###

housing.head(n = 5)

,Order,Lot_Area,Overall_Qual,Overall_Cond,Mas_Vnr_Area,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Full_Bath,Half_Bath,Kitchen_AbvGr,TotRms_AbvGr,Fireplaces,Garage_Cars,Garage_Area,Porch_Area,Pool_Area,Sale_Price,log_Sale_Price,log_Lot_Area,log_Mas_Vnr_Area,m_Mas_Vnr_Area,m_Total_Bsmt_SF,m_Garage_Cars,m_Garage_Area,m_log_Mas_Vnr_Area,has_Second_Flr,has_Garage,has_Mas_Vnr,has_Porch,log_Overall_Qual,Total_Bath,Grvl,Pave,Corner,CulDSac,FR2,FR3,Inside,Blmngtn,Blueste,BrDale,BrkSide,ClearCr,CollgCr,Crawfor,Edwards,Gilbert,Greens,GrnHill,IDOTRR,Landmrk,MeadowV,Mitchel,NAmes,NPkVill,NWAmes,NoRidge,NridgHt,OldTown,SWISU,Sawyer,SawyerW,Somerst,StoneBr,Timber,Veenker
0,1,31770,6,5,112,1080,1656,0,1656,1,0,1,7,2,2,528,272,0,215000,12.278393,10.366278,4.718508,0,0,0,0,0,0,1,1,1,1.791759,1.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2,11622,5,6,0,882,896,0,896,1,0,1,5,0,1,730,260,0,105000,11.561716,9.360655,-6.907755,0,0,0,0,0,0,1,0,1,1.609438,1.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,3,14267,6,6,108,1329,1329,0,1329,1,1,1,6,0,1,312,429,0,172000,12.055250,9.565704,4.682140,0,0,0,0,0,0,1,1,1,1.791759,1.5,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,11160,7,5,0,2110,2110,0,2110,2,1,1,8,2,2,522,0,0,244000,12.404924,9.320091,-6.907755,0,0,0,0,0,0,1,0,0,1.945910,2.5,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,5,13830,5,5,0,928,928,701,1629,2,1,1,6,1,2,482,246,0,189900,12.154253,9.534595,-6.907755,0,0,0,0,0,1,1,0,1,1.609438,2.5,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 2: Train-Test Split</h3>
Set up train-test split in the cell below. Set your <em>test_size</em> to 0.25 and your <em>random_state</em> to 219.

In [5]:
# preparing x-variables
# x_data = housing.loc[ : , reduced_x ] # x-variables (can change this)
x_data = housing.drop(['Sale_Price', 'log_Sale_Price'], axis = 1)


# preparing y-variable
y_data = housing.loc[ : , 'Sale_Price']    # y-variable
#y_data = housing.loc[ : , 'logSale_Price'] # y-variable


# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            x_data,
            y_data,
            test_size    = 0.25,
            random_state = 219 )


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 3: Check Available Hyperparameters</h3>
Call help(&nbsp;) on your selected model types to see which hyperparameters are available for tuning. Do NOT tune random_state, n_jobs or anything related to the intercept of a model. Use as many cells as needed.

In [6]:
#calling help() on RandomForestRegressor models
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble._forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimato

In [36]:
#calling help() on GradientBoostingRegressor models
help(GradientBoostingRegressor)

Help on class GradientBoostingRegressor in module sklearn.ensemble._gb:

class GradientBoostingRegressor(sklearn.base.RegressorMixin, BaseGradientBoosting)
 |  GradientBoostingRegressor(*, loss='squared_error', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
 |  
 |  Gradient Boosting for regression.
 |  
 |  GB builds an additive model in a forward stage-wise fashion;
 |  it allows for the optimization of arbitrary differentiable loss functions.
 |  In each stage a regression tree is fit on the negative gradient of the
 |  given loss function.
 |  
 |  Read more in the :ref:`User Guide <gradient_boosting>`.
 |  
 |  Parameters
 |  ----------
 |  loss : {'squared_er

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 4: Model Development - Default Hyperparameters</h3>
Run each of your selected models using three (3) respective default hyperparameters that you feel would be good candidates for optimization, printing their R-Square values for the training and testing sets. Use as many cells as needed and make sure to label your default hyperparameters.

<h4> Random Forest Regressor </h4>

Instantiating a random forest model using default hyperparameters

In [54]:
# specifying a model name
model_name = 'Unpruned Random Forest'


# INSTANTIATING a random forest model with default values
model = RandomForestRegressor(n_estimators     = 100,
                              criterion        = 'squared_error',
                              max_depth        = None,
                              min_samples_leaf = 1,
                              bootstrap        = True,
                              warm_start       = False,
                              random_state     = 219,
                              verbose          = 0,
                              ccp_alpha        = 0.0,
                              max_features     = 'auto')

# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print("Model Number   : 1")
print(f"Model Name     : {model_name}")
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

Model Number   : 1
Model Name    : Unpruned Random Forest
Training Score : 0.9829
Testing Score  : 0.873
Train-Test Gap : 0.1099


<h4> Gradient Boosted Machine </h4>



In [53]:
# specifying a model name
model_name = 'Unpruned GBM'


# INSTANTIATING the model object without hyperparameters
model = GradientBoostingRegressor(loss          = 'ls',
                                  learning_rate = 0.1,
                                  n_estimators  = 100,
                                  criterion     = 'friedman_mse',
                                  max_depth     = 3,
                                  warm_start    = False,
                                  random_state  = 219)


# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print("Model Number   : 2")
print(f"Model Name    : {model_name}")
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

/Users/udhayaa_t/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:286: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  warnings.warn(


Model Number   : 2
Model Name     : Unpruned GBM
Training Score : 0.9482
Testing Score  : 0.8913
Train-Test Gap : 0.0569


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 5: Hyperparameter Tuning Part I</h3><br>
Develop ranges for each of the hyperparameters you would like tune. Write a code to calculate how many models you are building (based on the combinations of hyperparameters you are tuning).
<br><br>
<strong>Requirement:</strong> You must tune at least three hyperparameters per model.<br>
<strong>Recommendation:</strong> Keep the number of models (i.e., iterations) you are building to:

* Less than 2,000 for Random Forest and GBM  (not including cross-validation)
* Less than 10,000 for all other model types (not including cross-validation)

Insert as many code cells as you need for the number of model types you are tuning.

<h4> Part 1 Tuning for <em>Random Forest Regression:</em> </h4>
    <ul>Setting <strong>4</strong> parameters for tuning<br><br>
    <table>
        <tr>
        <th>Name</th>
        <th>Parameter</th>
        </tr>
        <tr>
        <td>criterion</td>
        <td>"squared_error", "absolute_error", and "poisson"</td>
        </tr>
        <tr>
        <td>max_depth</td>
        <td>6 to 8</td>
        </tr>
        <tr>
        <td>min_samples_leaf</td>
        <td>1 to 4</td>
        </tr>
        <tr>
        <td>min_samples_split</td>
        <td>19 to 22</td>
        </tr>
        </table>
    </ul>
    <br>
    These parameters are revised parameters of second discovery. <br>The second parameter had a combination of 1200+ and<br> took more than 2 hours to run the single cell.

In [73]:
# declaring a hyperparameter space
criterion_range = ["squared_error", "absolute_error", "poisson"]
# splitter_range  = ['best', 'random']
depth_range   = np.arange(6, 9, 1)
leaf_range  = np.arange(1, 5, 1)
sample_split_range = np.arange(19,23,1)

#Displaying model name and number
print("Model Number      : 1\n"
      "Model Name        : Random Forest Regressor\n"
      "Hyperparameter    : 4\n")
#printing combination of hyperparameter tuning
print("Hyperparameter Tuning Part I total combination:",
      len(criterion_range) * 
      len(depth_range) * 
      len(leaf_range)*
      len(sample_split_range))


Model Number      : 1
Model Name        : Random Forest Regressor
Hyperparameter    : 4

Hyperparameter Tuning Part I total combination: 144


<h4> Part 1 Tuning for <em>Gradient Boosted Machine:</em> </h4>
    <ul>Setting <strong>9</strong> parameters for tuning <br><br>
    <table>
    <tr>
    <th>Name</th>
    <th>Parameter</th>
    </tr>
    <tr>
    <td>criterion</td>
    <td>'friedman_mse' and 'squared_error'</td>
    </tr>
    <tr>
    <td>max_depth</td>
    <td>6 to 8</td>
    </tr>
    <tr>
    <td>min_samples_leaf</td>
    <td>4 and 5</td>
    </tr>
    <tr>
    <td>min_samples_split</td>
    <td>7 and 8</td>
    </tr>
    <tr>
    <td>n_estimators</td>
    <td>180, 200, and 220</td>
    </tr>
    <tr>
    <td>loss</td>
    <td>'squared_error' and 'huber'</td>
    </tr>
    <tr>
    <td>learning_rate</td>
    <td>0.1</td>
    </tr>
    <tr>
    <td>alpha</td>
    <td>0.1 and 0.2</td>
    </tr>
    <tr>
    <td>ccp_alpha</td>
    <td>0.1</td>
    </tr>
    </table>
    </table><br>
        <br>
    These parameters are revised parameters of third discovery. <br>The third parameter had a combination of 1700+ and<br> took more than 3+ hours to running the single cell.

In [64]:
# Model 2 - Gradient Boosting Regressor
# Parameter 3

# declaring a hyperparameter space
criterion_range_2_para_4 = ['friedman_mse', 'squared_error']
loss_range_2_para_4 = ['squared_error', 'huber']
learning_rate_range_2_para_4 = [x/10 for x in range(1,2)]
depth_range_2_para_4     = range(6, 9, 1)
leaf_range_2_para_4      = range(4, 6, 1)
sample_split_range_2_para_4 = range(7,9,1)
n_estimators_range_2_para_4 = range(180,221,20)
alpha_range_2_para_4 = [x/10 for x in range(1,3)]
ccp_alpha_range_2_para_4 = [x/10 for x in range(1,2)]

#Displaying model name and number
print("Model Number      : 2\n"
      "Model Name        : Gradient Boosting Regressor\n"
      "Hyperparameter    : 9\n"
      "Total Combination :", #printing combination of hyperparameter tuning
      len(criterion_range_2_para_4) * 
      len(depth_range_2_para_4) * 
      len(leaf_range_2_para_4) *
      len(sample_split_range_2_para_4)*
      len(n_estimators_range_2_para_4) *
      len(loss_range_2_para_4) * 
      len(alpha_range_2_para_4) *
      len(ccp_alpha_range_2_para_4) *
      len(learning_rate_range_2_para_4))

Model Number      : 2
Model Name        : Gradient Boosting Regressor
Hyperparameter    : 9
Total Combination : 288


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 6: Hyperparameter Tuning Part II</h3><br>
Create a hyperparameter grid for each of the models you are tuning. Note that this is the dictionary step we conducted in class. Use as many cells as needed for this task.

In [65]:
# creating a hyperparameter grid
param_grid = {'criterion'        : criterion_range,
              'max_depth'        : depth_range,
              'min_samples_leaf' : leaf_range,
              'min_samples_split': sample_split_range}

In [66]:
# creating a hyperparameter grid
param_grid_2_para_4 = {'criterion'        : criterion_range_2_para_4,
                       'max_depth'        : depth_range_2_para_4,
                       'min_samples_leaf' : leaf_range_2_para_4,
                       'min_samples_split': sample_split_range_2_para_4,
                       'n_estimators'     : n_estimators_range_2_para_4,
                       'loss'             : loss_range_2_para_4,
                       'learning_rate'    : learning_rate_range_2_para_4,
                       'alpha'            : alpha_range_2_para_4,
                       'ccp_alpha'        : ccp_alpha_range_2_para_4}

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 7: Hyperparameter Tuning Part III</h3><br>
a) Complete the remaining steps for hyperparameter tuning. While your code runs, observe the processing time it takes to tune. Use as many cells as needed for this task.

In [74]:

# INSTANTIATING the model object without hyperparameters
model = RandomForestRegressor(random_state = 219)



# RandomizedSearchCV object
model_cv = RandomizedSearchCV(estimator           = model,
                              param_distributions = param_grid,
                              random_state        = 219)



# FITTING to the FULL DATASET (due to cross-validation)
model_cv.fit(x_data, y_data)



# printing the optimal parameters and best score
print(f"Tuned Parameters: {model_cv.best_params_}")
print(f"Tuned R-Square:   {model_cv.best_score_}")



/Users/udhayaa_t/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/udhayaa_t/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/udhayaa_t/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/Users/udhayaa_t/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(it

Tuned Parameters: {'min_samples_split': 9, 'min_samples_leaf': 2, 'max_depth': 8, 'criterion': 'absolute_error'}
Tuned R-Square:   0.8625568533267955


In [67]:
# INSTANTIATING the model object without hyperparameters
model_2_para_4 = GradientBoostingRegressor(random_state = 219)

# RandomizedSearchCV object
model_2_para_4_cv = RandomizedSearchCV(estimator           = model_2_para_4,
                                       param_distributions = param_grid_2_para_4,
                                       n_iter              = 1000,
                                       cv                  = 5,
                                       random_state        = 219)

# FITTING to the FULL DATASET (due to cross-validation)
model_2_para_4_cv.fit(x_data, y_data)

# printing the optimal parameters and best score
print(f"Tuned Parameters: {model_2_para_4_cv.best_params_}")
print(f"Tuned R-Square:   {model_2_para_4_cv.best_score_}")



/Users/udhayaa_t/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 288 is smaller than n_iter=1000. Running 288 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Tuned Parameters: {'n_estimators': 220, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_depth': 6, 'loss': 'huber', 'learning_rate': 0.1, 'criterion': 'squared_error', 'ccp_alpha': 0.1, 'alpha': 0.2}
Tuned R-Square:   0.8877473864005212


<br>
b) Explore the hyperparameter tuning results for one of your tuned models. Look for hyperparameter combinations that tied for first place. Test each of these models using train-test split. If no models tied for first place, test your top three models.

In [75]:
# Defining function for tuning
def tuning_results(cv_results, n=5):
    """
This function will display the top "n" models from hyperparameter tuning,
based on "rank_test_score".

PARAMETERS
----------
cv_results = results dictionary from the attribute ".cv_results_"
n          = number of models to display
    """
    param_lst = [] #Empty list

    for result in cv_results["params"]: #for loop to append 
        result = str(result).replace(":", "=")
        param_lst.append(result[1:-1])


    results_df = pd.DataFrame(data = {
        "Model_Rank" : cv_results["rank_test_score"],
        "Mean_Test_Score" : cv_results["mean_test_score"],
        "SD_Test_Score" : cv_results["std_test_score"],
        "Parameters" : param_lst
    })


    results_df = results_df.sort_values(by = "Model_Rank", axis = 0)
    return results_df.head(n = n)



# run tuning_results() on the hyperparameter tuning results
print(tuning_results(cv_results = model_cv.cv_results_, n = 5))
# run tuning_results() on the hyperparameter tuning results
print(tuning_results(cv_results = model_2_para_4_cv.cv_results_, n = 5))

   Model_Rank  Mean_Test_Score  SD_Test_Score                                                                                    Parameters
8           1         0.862557       0.033812  'min_samples_split'= 9, 'min_samples_leaf'= 2, 'max_depth'= 8, 'criterion'= 'absolute_error'
5           2         0.862386       0.032590  'min_samples_split'= 6, 'min_samples_leaf'= 1, 'max_depth'= 8, 'criterion'= 'absolute_error'
1           3         0.862376       0.034663   'min_samples_split'= 8, 'min_samples_leaf'= 2, 'max_depth'= 8, 'criterion'= 'squared_error'
3           4         0.861543       0.034576  'min_samples_split'= 2, 'min_samples_leaf'= 3, 'max_depth'= 8, 'criterion'= 'absolute_error'
0           5         0.858998       0.032034   'min_samples_split'= 8, 'min_samples_leaf'= 1, 'max_depth'= 7, 'criterion'= 'squared_error'
     Model_Rank  Mean_Test_Score  SD_Test_Score                                                                                           Parameters
257        

<h4>Pruned Random Forest</h4>
    Testing the top three models for Pruned Random Forest and the variation is because of the train-test gap between the models.

In [76]:
# specifying a model name
model_name = 'Pruned Random Forest'


# INSTANTIATING a random forest model with default values
model = RandomForestRegressor(n_estimators      = 100,
                              criterion         = 'absolute_error',
                              max_depth         = 8,
                              min_samples_leaf  = 2,
                              min_samples_split = 9,
                              random_state      = 219)


# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print("Model Name     : Pruned Random Forest\n")
print("Model Rank     : 1")
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

Model Name     : Pruned Random Forest

Training Score : 0.938
Testing Score  : 0.8716
Train-Test Gap : 0.0664


In [77]:
# specifying a model name
model_name = 'Pruned Random Forest'


# INSTANTIATING a random forest model with default values
model = RandomForestRegressor(n_estimators      = 100,
                              criterion         = 'absolute_error',
                              max_depth         = 8,
                              min_samples_leaf  = 1,
                              min_samples_split = 6,
                              random_state      = 219)


# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print("Model Name     : Pruned Random Forest\n")
print("Model Rank     : 2")
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

Model Name     : Pruned Random Forest

Model Rank     : 2
Training Score : 0.9472
Testing Score  : 0.87
Train-Test Gap : 0.0772


In [78]:
# specifying a model name
model_name = 'Pruned Random Forest'


# INSTANTIATING a random forest model with default values
model = RandomForestRegressor(n_estimators      = 100,
                              criterion         = 'squared_error',
                              max_depth         = 8,
                              min_samples_leaf  = 2,
                              min_samples_split = 8,
                              random_state      = 219)


# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print("Model Name     : Pruned Random Forest\n")
print("Model Rank     : 3")
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

Model Name     : Pruned Random Forest

Model Rank     : 3
Training Score : 0.9457
Testing Score  : 0.8708
Train-Test Gap : 0.0749


In [ ]:
# Final model
# specifying a model name
model_name = 'Pruned Random Forest'


# INSTANTIATING a random forest model with default values
# the following parameters has been tuned to the highest score 
# using the top 5 tuning result
model = RandomForestRegressor(criterion        = 'absolute_error',
                              max_depth        = 8, 
                              min_samples_leaf = 3,
                              min_samples_split = 21,
                              random_state     = 219)


# FITTING the training data
model_fit = model.fit(x_train, y_train)


# PREDICTING based on the testing set
model_pred = model.predict(x_test)


# SCORING the results
model_train_score = model.score(x_train, y_train).round(4) # using R-square
model_test_score  = model.score(x_test, y_test).round(4)   # using R-square
model_gap         = abs(model_train_score - model_test_score).round(4)


# displaying results
print("Model Name     : Pruned Random Forest\n")
print('Training Score :', model_train_score)
print('Testing Score  :', model_test_score)
print('Train-Test Gap :', model_gap)

<h4>Gradient Boosting: </h4>
        Two models tied for the first place and exploring both the models

In [80]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
model_2_para_4_tune = GradientBoostingRegressor(min_samples_split = 8,
                                            min_samples_leaf = 4,
                                            max_depth        = 6,
                                            criterion        = 'squared_error',
                                            random_state     = 219,
                                            n_estimators= 220,
                                            loss = 'huber',
                                            learning_rate = 0.1,
                                            alpha = 0.2,
                                            ccp_alpha = 0.1)


# FITTING to the TRAINING data
model_2_para_4_tune_fit = model_2_para_4_tune.fit(x_train, y_train)


# PREDICTING based on the testing set
model_2_para_4_tune_pred = model_2_para_4_tune.predict(x_test)


# SCORING the results
model_2_para_4_tune_train_score = model_2_para_4_tune.score(x_train, y_train).round(4) # using R-square
model_2_para_4_tune_test_score  = model_2_para_4_tune.score(x_test, y_test).round(4)   # using R-square
model_2_para_4_tune_gap         = abs(model_2_para_4_tune_train_score - model_2_para_4_tune_test_score).round(4)


# displaying results
print("Model Name     : Gradient Boosting Regressor\n")
print('Training Score :', model_2_para_4_tune_train_score)
print('Testing Score  :', model_2_para_4_tune_test_score)
print('Train-Test Gap :', model_2_para_4_tune_gap)

Model Name     : Gradient Boosting Regressor

Training Score : 0.9704
Testing Score  : 0.888
Train-Test Gap : 0.0824


In [79]:
# INSTANTIATING a logistic regression model with tuned values
model_2_para_4_tune = GradientBoostingRegressor(min_samples_split = 7,
                                            min_samples_leaf = 4,
                                            max_depth        = 6,
                                            criterion        = 'squared_error',
                                            random_state     = 219,
                                            n_estimators= 220,
                                            loss = 'huber',
                                            learning_rate = 0.1,
                                            alpha = 0.2,
                                            ccp_alpha = 0.1)


# FITTING to the TRAINING data
model_2_para_4_tune_fit = model_2_para_4_tune.fit(x_train, y_train)


# PREDICTING based on the testing set
model_2_para_4_tune_pred = model_2_para_4_tune.predict(x_test)


# SCORING the results
model_2_para_4_tune_train_score = model_2_para_4_tune.score(x_train, y_train).round(4) # using R-square
model_2_para_4_tune_test_score  = model_2_para_4_tune.score(x_test, y_test).round(4)   # using R-square
model_2_para_4_tune_gap         = abs(model_2_para_4_tune_train_score - model_2_para_4_tune_test_score).round(4)


# displaying results
print("Model Name     : Gradient Boosting Regressor\n")
print('Training Score :', model_2_para_4_tune_train_score)
print('Testing Score  :', model_2_para_4_tune_test_score)
print('Train-Test Gap :', model_2_para_4_tune_gap)

Model Name     : Gradient Boosting Regressor

Training Score : 0.9704
Testing Score  : 0.888
Train-Test Gap : 0.0824


In [81]:
#Final Model

# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
# the following parameters has been tuned to the highest score 
# using the top 5 tuning result
model_2_para_4_tune = GradientBoostingRegressor(min_samples_split = 9,
                                            min_samples_leaf = 9,
                                            max_depth        = 5,
                                            criterion        = 'squared_error',
                                            random_state     = 219,
                                            n_estimators= 120,
                                            loss = 'huber',
                                            learning_rate = 0.1,
                                            alpha = 0.2,
                                            ccp_alpha = 0.1)


# FITTING to the TRAINING data
model_2_para_4_tune_fit = model_2_para_4_tune.fit(x_train, y_train)


# PREDICTING based on the testing set
model_2_para_4_tune_pred = model_2_para_4_tune.predict(x_test)


# SCORING the results
model_2_para_4_tune_train_score = model_2_para_4_tune.score(x_train, y_train).round(4) # using R-square
model_2_para_4_tune_test_score  = model_2_para_4_tune.score(x_test, y_test).round(4)   # using R-square
model_2_para_4_tune_gap         = abs(model_2_para_4_tune_train_score - model_2_para_4_tune_test_score).round(4)


# displaying results
print("Model Name     : Gradient Boosting Regressor\n")
print('Training Score :', model_2_para_4_tune_train_score)
print('Testing Score  :', model_2_para_4_tune_test_score)
print('Train-Test Gap :', model_2_para_4_tune_gap)

Model Name     : Gradient Boosting Regressor

Training Score : 0.9327
Testing Score  : 0.8916
Train-Test Gap : 0.0411


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<h3>Step 8: (optional) Explore Tuned Hyperparameters</h3><br>
After tuning, it is a great idea to explore any hyperparameters that were markedly different from their default values once tuned. Investigate the purpose of these hyperparameters in <a href="https://scikit-learn.org/stable/index.html">the scikit-learn documentation</a>.